In [1]:
import math
import numpy as np
import torch
import pkbar
from unet3d.config import *
from utils.Visualization import ImageSliceViewer3D
from tqdm import tqdm
from torch.nn import CrossEntropyLoss
import glob
from torch.optim import Adam
from unet3d.dataset import SAIADDataset
from torch.utils.data import DataLoader
from patchify import patchify
from utils.Other import get_headers
import nrrd
from unet3d.transforms import train_transform_cuda

%load_ext autoreload
%autoreload 2
%matplotlib inline
_,_,patient_names = get_headers(DATASET_PATH)
device = 'mps'


In [2]:
from unet3d.dataset import WrappedDataLoader, to_device


excl_patients_training = ['SAIAD 15', 'SAIAD 11'] #patients for validation/testing
excl_patients_val = list(set(patient_names) - set(excl_patients_training))

print("Training with val patients:", excl_patients_training)



## Load dataset ##
train_dataset = SAIADDataset(
    excl_patients=excl_patients_training,
    load_data_to_memory=True,
    n_batches=TRAIN_BATCHES_PER_EPOCH,
    transform=None,
    non_unif_sampling=False
    )
val_dataset = SAIADDataset(
    excl_patients=excl_patients_val,
    load_data_to_memory=True,
    n_batches=VAL_BATCHES_PER_EPOCH,
)

train_dataloader = DataLoader(
    train_dataset, 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=False, 
    pin_memory=False, 
    num_workers=NUM_WORKERS
    )
val_dataloader = DataLoader(
    val_dataset, 
    batch_size=VAL_BATCH_SIZE,
    shuffle=False, 
    pin_memory=False, 
    num_workers=NUM_WORKERS
    )

train_dataloader = WrappedDataLoader(train_dataloader, to_device, device)

Training with val patients: ['SAIAD 15', 'SAIAD 11']
Fetching patients probabilities...


100%|██████████| 2/2 [00:00<00:00, 13.90it/s]


In [3]:
for epoch in tqdm(range(EPOCHS)):
    for X_batch, y_batch in train_dataloader:
        pass


  1%|          | 1/150 [00:31<1:19:14, 31.91s/it]


KeyboardInterrupt: 

In [4]:
X_batch.dtype

torch.float16

In [59]:
n=0
ImageSliceViewer3D(np.array(patch_scan[0]), np.array(torch.argmax(patch_segm, dim=0)))

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [79]:
from monai.transforms import Compose, RandFlipd, ToTensord, RandGaussianNoised, RandAdjustContrastd
patchs = {'name': 'patches', 'patch_scan': patch_scan, 'patch_segm': patch_segm}
transform = Compose(
    [   
        RandFlipd(keys=['patch_scan', 'patch_segm'], prob=1, spatial_axis=0),
        RandGaussianNoised(keys=['patch_scan', 'patch_segm'], prob=1, mean=0.0, std=0.1),
        RandAdjustContrastd(keys=['patch_scan', 'patch_segm'], prob=1, gamma=(0.5,2)),
        ToTensord(keys=['patch_scan', 'patch_segm'], device='mps')
    ]
)
trans_patchs = transform(patchs)
ImageSliceViewer3D(np.array(trans_patchs['patch_scan'].cpu()[0]), np.array(patch_scan[0]))

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [23]:
patch_scan.max()

tensor(255.)